In [1]:
# requires (unimported) openpyxl
import pandas as pd
import sqlite3

import scoped_mapping

In [2]:
# https://github.com/microbiomedata/nmdc-metadata/issues/371#event-4949651397
# https://docs.google.com/spreadsheets/d/1WErXj8sM5uJi51VVLNQZDilDF7wMiyBC2T4zELp7Axc/edit#gid=1239761730

package_review_blm_fp = '../../data/EnvO_triad_terms_MIxS_soil_package_review_05182021.xlsx'

package_review_vs_INSDC_fp = "../../target/package_review_vs_INSDC.tsv"

biosample_sqlite_file = "../../target/harmonized-table.db"

biosample_cnx = sqlite3.connect(biosample_sqlite_file)

In [3]:
# implicitly opens first sheet "Subset_EnvO_Broad_Local_Medium_"
package_review_blm = pd.read_excel(
     package_review_blm_fp, engine='openpyxl'
)

In [4]:
package_review_blm.columns=['class','label','definition','packages','extra']
package_review_blm['slot'] = package_review_blm.index
package_review_blm = package_review_blm[['slot','class','label','packages']]
package_review_blm = package_review_blm.reset_index(drop=True)

# convert class IRIs to CURIEs... all envo?
# not the best approach by a long shot
package_review_blm['class'] = package_review_blm['class'].str.replace('<http://purl.obolibrary.org/obo/ENVO_', 'ENVO:', regex=False)
package_review_blm['class'] = package_review_blm['class'].str.replace('>', '', regex=False)
package_review_blm = package_review_blm.sort_values(by = ['class'])

package_review_blm


,slot,class,label,packages
103,env_local_scale,ENVO:00000011,garden,Soil|Plant-associated
86,env_local_scale,ENVO:00000014,canal,Water
115,env_local_scale,ENVO:00000020,lake,Sediment|Water
136,env_local_scale,ENVO:00000022,river,Sediment|Water
144,env_local_scale,ENVO:00000023,stream,Water
...,...,...,...,...
239,env_medium,ENVO:01001841,volcanic soil,Soil
92,env_local_scale,ENVO:02000049,coastal water body,Water
229,env_medium,ENVO:02000059,surface soil,Soil
89,env_local_scale,ENVO:03000117,channel,Soil|Sediment|Water


In [5]:
package_review_blm['packages'].value_counts()

Soil                              141
Water                              43
Sediment|Water                     15
Water|Sediment                      8
Sediment                            6
Soil|Water                          5
Soil|Sediment|Water                 4
Water|Host-associted                3
Soil|Sediment                       3
HCR-cores|HCR-fluids-swabs          3
Soil|Plant-associated               3
Water|Host-associated               2
Soil|Sediment|Plant-associated      1
Water|Air                           1
Built-environment                   1
Host-associated                     1
Plant-associated                    1
Name: packages, dtype: int64

In [6]:
miniframes = []
for row in package_review_blm.itertuples(index=True, name='Pandas'):
    tuple_index = row[0]
    rp = row.packages
    rps = rp.split("|")
    rpss = pd.Series(rps)
    rps_len = len(rps)
    index_rep = [tuple_index] * rps_len
    irs = pd.Series(index_rep)
    temp = pd.concat([irs, rpss], axis=1)
    miniframes.append(temp)
    
maxiframe = pd.concat(miniframes)

maxiframe.columns = ['saved_index','split_val']

In [7]:
maxiframe

,saved_index,split_val
0,103,Soil
1,103,Plant-associated
0,86,Water
0,115,Sediment
1,115,Water
...,...,...
0,89,Soil
1,89,Sediment
2,89,Water
0,80,Water


In [8]:
merged = package_review_blm.merge(maxiframe, left_index=True, right_on='saved_index')

In [9]:
merged = merged[['split_val','slot','class','label']]

merged.columns = ['package','slot','class','label']

merged['package'] = merged['package'].str.lower()
merged['label'] = merged['label'].str.lower()

merged

,package,slot,class,label
0,soil,env_local_scale,ENVO:00000011,garden
1,plant-associated,env_local_scale,ENVO:00000011,garden
0,water,env_local_scale,ENVO:00000014,canal
0,sediment,env_local_scale,ENVO:00000020,lake
1,water,env_local_scale,ENVO:00000020,lake
...,...,...,...,...
0,soil,env_local_scale,ENVO:03000117,channel
1,sediment,env_local_scale,ENVO:03000117,channel
2,water,env_local_scale,ENVO:03000117,channel
0,water,env_local_scale,ENVO:2000004,algal bloom


In [10]:
merged.to_csv("package_review_blm_long.tsv", sep='\t', index=False)

In [11]:
q = """
select
	scoping_value ,
	biosample_col_to_map,
	consensus_id ,
	consensus_lab,
	count(1) as count
from
	repaired_long rl
group by
	scoping_value ,
	biosample_col_to_map,
	consensus_id ,
	consensus_lab
order by
	scoping_value ,
	biosample_col_to_map,
	consensus_lab;
"""
[insdc_mixs_envo_usage, query_duration] = scoped_mapping.timed_query(q, biosample_cnx)

print(query_duration)

0:00:00.186243


In [12]:
insdc_mixs_envo_usage

,scoping_value,biosample_col_to_map,consensus_id,consensus_lab,count
0,plant-associated,env_broad_scale,ENVO:00000114,agricultural field,220
1,plant-associated,env_broad_scale,ENVO:01000245,cropland biome,825
2,plant-associated,env_broad_scale,ENVO:01000311,cultivated environment,1
3,plant-associated,env_broad_scale,ENVO:00002011,fresh water,50
4,plant-associated,env_broad_scale,ENVO:00000873,freshwater biome,57
...,...,...,...,...,...
711,water,env_medium,ENVO:00002001,waste water,3
712,water,env_medium,CHEBI:15377,water,4301
713,water,env_medium,ENVO:06105011,water column,40
714,water,env_medium,ENVO:01000277,water ice,5


In [13]:
package_review_vs_INSDC = insdc_mixs_envo_usage.merge(merged, 
                                                      how = "outer", 
                                                      left_on=['consensus_id', 'scoping_value', 'biosample_col_to_map'], 
                                                      right_on=['class', 'package', 'slot'])

package_review_vs_INSDC['reccomended'] = ~ package_review_vs_INSDC['class'].isna()
package_review_vs_INSDC['package'].fillna(package_review_vs_INSDC['scoping_value'], inplace=True)
package_review_vs_INSDC['slot'].fillna(package_review_vs_INSDC['biosample_col_to_map'], inplace=True)
package_review_vs_INSDC['class'].fillna(package_review_vs_INSDC['consensus_id'], inplace=True)
package_review_vs_INSDC['label'].fillna(package_review_vs_INSDC['consensus_lab'], inplace=True)
package_review_vs_INSDC['count'].fillna(0, inplace=True)

package_review_vs_INSDC = package_review_vs_INSDC[['package', 'slot', 'class', 'label', 'count', 'reccomended']]

package_review_vs_INSDC = package_review_vs_INSDC.sort_values(by = ['reccomended','count'], ascending = [False, False])

In [14]:
package_review_vs_INSDC

,package,slot,class,label,count,reccomended
486,water,env_broad_scale,ENVO:00000447,marine biome,3680.0,True
506,water,env_broad_scale,ENVO:01000048,ocean biome,1498.0,True
470,water,env_broad_scale,ENVO:00000873,freshwater biome,1068.0,True
292,soil,env_broad_scale,ENVO:01000249,urban biome,1011.0,True
291,soil,env_broad_scale,ENVO:01000180,tundra biome,524.0,True
...,...,...,...,...,...,...
678,water,env_medium,ENVO:00000020,lake,1.0,False
685,water,env_medium,ENVO:01001038,moderately saline water,1.0,False
686,water,env_medium,ENVO:00000569,obsolete marine habitat,1.0,False
688,water,env_medium,ENVO:00002227,obsolete saline water habitat,1.0,False


In [15]:
package_review_vs_INSDC.to_csv(package_review_vs_INSDC_fp, sep='\t', index=False)